
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/monacofj/moeabench/blob/main/examples/example_10.ipynb)

# Example 10: Topological Equivalence (topo_dist)

This example demonstrates how to use the 'topo_dist' tool to determine if two different algorithms have converged to statistically equivalent distributions in both the objective and decision spaces.

In [ ]:
!pip install --quiet git+https://github.com/monacofj/moeabench.git

In [ ]:
from MoeaBench import mb
import matplotlib.pyplot as plt
print(f"Version: {mb.system.version()}")

# 1. Setup: Compare NSGA-II vs NSGA-III on DTLZ2
# We use a 3-objective problem (M=3)
exp1 = mb.experiment()
exp1.name = "NSGA-II"
exp1.mop = mb.mops.DTLZ2(M=3)
exp1.moea = mb.moeas.NSGA2(population=100, generations=50)

exp2 = mb.experiment()
exp2.name = "NSGA-III"
exp2.mop = mb.mops.DTLZ2(M=3)
exp2.moea = mb.moeas.NSGA3(population=100, generations=50)

# 2. Execution: Run both experiments
print("Running NSGA6-II...")
exp1.run()

print("Running NSGA-III...")
exp2.run()

In [ ]:
# 3. Topological Analysis: Objective Space
# By default, topo_distribution(exp1, exp2) uses space='objs' and alpha=0.05
print("\n--- [Analysis 1] Convergence Equivalence (Objective Space) ---")
res_objs = mb.stats.topo_distribution(exp1, exp2, alpha=0.01) # Stricter alpha
print(res_objs.report())

In [ ]:
# 4. Topological Analysis: Decision Space
# We can also check if they found the same solutions in the decision space
print("\n--- [Analysis 2] Strategy Equivalence (Decision Space) ---")
res_vars = mb.stats.topo_distribution(exp1, exp2, space='vars')
print(res_vars.report())

In [ ]:
# 5. Advanced: Earth Mover Distance (Geometric Distance)
# Quantify "how far" the distributions are from each other
print("\n--- [Analysis 3] Geometric Distance (EMD) ---")
# Custom threshold: declare match only if distance < 0.05
res_emd = mb.stats.topo_distribution(exp1, exp2, method='emd', threshold=0.05)
print(res_emd.report())

### 6. Visual Verification: Distribution Plots

We now visualize the probability densities to understand the statistical verdict. 

In [ ]:
# Visual 1: Objective Space Match (Convergence)
mb.view.topo_density(exp1, exp2, space='objs', axes=[0], alpha=0.01, 
                     title="Objective Match (f1) [alpha=0.01]")

In [ ]:
# Visual 2: Decision Space Mismatch (Strategy)
mb.view.topo_density(exp1, exp2, space='vars', axes=[2], alpha=0.05, 
                     title="Decision Mismatch (x3) [alpha=0.05]")

### Interpretation

1. **Objective Space Match**: The overlapping curves in the first plot confirm that both algorithms cover the same performance region.

2. **Decision Space Divergence**: The separated curves in the second plot reveal that they achieve this performance using different variable ranges.